크롤링 라이브러리 설치

In [ ]:
pip install pillow tqdm requests beautifulsoup4 ultralytics


bing 이미지 url 수집 함수

In [ ]:

breeds = ['Chihuahua', 'Japanese_spaniel', 'Maltese_dog', 'Pekinese', 'Shih-Tzu', 'Blenheim_spaniel', 'papillon', 'toy_terrier', 'Rhodesian_ridgeback', 'Afghan_hound', 'basset', 'beagle', 'bloodhound', 'bluetick', 'black-and-tan_coonhound', 'Walker_hound', 'English_foxhound', 'redbone', 'borzoi', 'Irish_wolfhound', 'Italian_greyhound', 'whippet', 'Ibizan_hound', 'Norwegian_elkhound', 'otterhound', 'Saluki', 'Scottish_deerhound', 'Weimaraner', 'Staffordshire_bullterrier', 'American_Staffordshire_terrier', 'Bedlington_terrier', 'Border_terrier', 'Kerry_blue_terrier', 'Irish_terrier', 'Norfolk_terrier', 'Norwich_terrier', 'Yorkshire_terrier', 'wire-haired_fox_terrier', 'Lakeland_terrier', 'Sealyham_terrier', 'Airedale', 'cairn', 'Australian_terrier', 'Dandie_Dinmont', 'Boston_bull', 'miniature_schnauzer', 'giant_schnauzer', 'standard_schnauzer', 'Scotch_terrier', 'Tibetan_terrier', 'silky_terrier', 'soft-coated_wheaten_terrier', 'West_Highland_white_terrier', 'Lhasa', 'flat-coated_retriever', 'curly-coated_retriever', 'golden_retriever', 'Labrador_retriever', 'Chesapeake_Bay_retriever', 'German_short-haired_pointer', 'vizsla', 'English_setter', 'Irish_setter', 'Gordon_setter', 'Brittany_spaniel', 'clumber', 'English_springer', 'Welsh_springer_spaniel', 'cocker_spaniel', 'Sussex_spaniel', 'Irish_water_spaniel', 'kuvasz', 'schipperke', 'groenendael', 'malinois', 'briard', 'kelpie', 'komondor', 'Old_English_sheepdog', 'Shetland_sheepdog', 'collie', 'Border_collie', 'Bouvier_des_Flandres', 'Rottweiler', 'German_shepherd', 'Doberman', 'miniature_pinscher', 'Greater_Swiss_Mountain_dog', 'Bernese_mountain_dog', 'Appenzeller', 'EntleBucher', 'boxer', 'bull_mastiff', 'Tibetan_mastiff', 'French_bulldog', 'Great_Dane', 'Saint_Bernard', 'Eskimo_dog', 'malamute', 'Siberian_husky', 'affenpinscher', 'basenji', 'pug', 'Leonberg', 'Newfoundland', 'Great_Pyrenees', 'Samoyed', 'Pomeranian', 'chow', 'keeshond', 'Brabancon_griffon', 'Pembroke', 'Cardigan', 'toy_poodle', 'miniature_poodle', 'standard_poodle', 'Mexican_hairless', 'dingo', 'dhole', 'African_hunting_dog']
# 위에 크롤링할 강아지 이름 넣으면됩니다
import requests
from bs4 import BeautifulSoup
import re
# bing에서 이미지 수집( google은 크롤링 막혀있음 )
def crawling_image_urls(query, max_count=200):
    query = query.replace(" ", "+")
    url = f"https://www.bing.com/images/search?q={query}&form=HDRSC2"

    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    urls = []
    img_tags = soup.find_all("a", class_="iusc")
    for tag in img_tags:
        try:
            m = re.search(r'"murl":"(.*?)"', tag.get("m"))
            if m:
                urls.append(m.group(1))
        except:
            pass
        if len(urls) >= max_count:
            break
    return urls



yolo crop function

In [ ]:
from ultralytics import YOLO
from PIL import Image
import torch

yolo = YOLO("yolov8n.pt")
device = "cuda" if torch.cuda.is_available() else "cpu"
yolo.to(device)

def crop_yolo(image):
    results = yolo(image)[0]
    dog_boxes = []
    for box in results.boxes:
        if int(box.cls) == 16:
            dog_boxes.append(box.xyxy[0].cpu().numpy())
    if len(dog_boxes) == 0:
        return None

    # 가장 큰 box를 select
    areas = [(b[2]-b[0])*(b[3]-b[1]) for b in dog_boxes]
    x1, y1, x2, y2 = map(int, dog_boxes[areas.index(max(areas))])
    return image.crop((x1, y1, x2, y2))


품종 리스트 돌리기

In [ ]:
import os
from tqdm import tqdm
from io import BytesIO

# target_count 수 만큼 품종 이미지를 Yolo로 잘라 이미지 크롤링
# max_count는 찾아볼 이미지의 수, 500에서 몇장 가져오지 못해서 1200으로 확장
def save_images(breed, target_count=50):
    save_dir = f"./cropped_{breed.replace(' ', '_')}"
    os.makedirs(save_dir, exist_ok=True)
    print(f"\n{breed} 이미지 다운로드 시작")
    urls = crawling_image_urls(breed + " dog", max_count=1200)
    print("URL 수집:", len(urls))

    count = 0
    for url in tqdm(urls):
        try:
            img_data = requests.get(url, timeout=5).content
            img = Image.open(BytesIO(img_data)).convert("RGB")
            cropped = crop_yolo(img)
            if cropped is None:
                continue
            if cropped.size[0] < 150 or cropped.size[1] < 150:
                continue
            filepath = os.path.join(save_dir, f"{breed}_{count}.jpg")
            cropped.save(filepath, "JPEG")
            count += 1
            if count >= target_count:
                break
        except:
            continue



In [ ]:
for breed in breeds:
    save_images(breed, target_count=100)



=== Chihuahua 이미지 다운로드 시작 ===
URL 수집: 35


  0%|          | 0/35 [00:00<?, ?it/s]


0: 480x640 1 dog, 217.9ms
Speed: 7.9ms preprocess, 217.9ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)


  3%|▎         | 1/35 [00:00<00:14,  2.31it/s]


0: 448x640 1 dog, 296.3ms
Speed: 4.9ms preprocess, 296.3ms inference, 1.9ms postprocess per image at shape (1, 3, 448, 640)


...


0: 544x640 1 bear, 212.8ms
Speed: 5.5ms preprocess, 212.8ms inference, 1.6ms postprocess per image at shape (1, 3, 544, 640)


 94%|█████████▍| 33/35 [00:14<00:00,  2.44it/s]


0: 480x640 1 bear, 196.6ms
Speed: 7.2ms preprocess, 196.6ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)


100%|██████████| 35/35 [00:14<00:00,  2.42it/s]

African_hunting_dog: 10장 저장 완료


In [ ]:
import os

base_dir = "/content/drive/MyDrive/dataset"

# dataset 폴더 내부 목록 가져오기
folders = sorted(os.listdir(base_dir))

for folder in folders:
    folder_path = os.path.join(base_dir, folder)

    # 폴더인지 확인
    if os.path.isdir(folder_path):
        # 이미지 파일(jpg, jpeg, png) 개수 세기
        count = sum(
            1 for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        )
        print(f"{folder}: {count}장")


cropped_Afghan_hound: 15장
cropped_African_hunting_dog: 10장
cropped_Airedale: 33장
cropped_American_Staffordshire_terrier: 22장
cropped_Appenzeller: 18장
cropped_Australian_terrier: 0장
cropped_Bedlington_terrier: 15장
cropped_Bernese_mountain_dog: 30장
cropped_Blenheim_spaniel: 31장
cropped_Border_collie: 0장
cropped_Border_terrier: 6장
cropped_Boston_bull: 0장
cropped_Bouvier_des_Flandres: 25장
cropped_Brabancon_griffon: 29장
cropped_Brittany_spaniel: 0장
cropped_Cardigan: 0장
cropped_Chesapeake_Bay_retriever: 16장
cropped_Chihuahua: 23장
cropped_Dandie_Dinmont: 22장
cropped_Doberman: 21장
cropped_English_foxhound: 25장
cropped_English_setter: 0장
cropped_English_springer: 0장
cropped_EntleBucher: 25장
cropped_Eskimo_dog: 0장
cropped_French_bulldog: 0장
cropped_German_shepherd: 23장
cropped_German_short-haired_pointer: 0장
cropped_Gordon_setter: 19장
cropped_Great_Dane: 0장
cropped_Great_Pyrenees: 0장
cropped_Greater_Swiss_Mountain_dog: 21장
cropped_Ibizan_hound: 22장
cropped_Irish_setter: 20장
cropped_Irish_terrier